![alt text](http://pandas.pydata.org/_static/pandas_logo.png)

<center><h1> PANDAS TABLE OF CONTENTS </h1></center>

## [Reshaping and Pivot Tables](http://pandas.pydata.org/pandas-docs/stable/reshaping.html) 


- [Reshaping by pivoting DataFrame objects](#Reshaping-by-pivoting-DataFrame-objects)
- [Reshaping by stacking and unstacking](#Reshaping-by-stacking-and-unstacking)
- [Reshaping by Melt](#Reshaping-by-Melt)
- [Combining with stats and GroupBy](#Combining-with-stats-and-GroupBy)
- [Pivot tables](#Pivot-tables)
- [Cross tabulations](#Cross-tabulations)
- [Tiling](#Tiling)
- [Factorizing values](#Factorizing-values)

In [ ]:
import pandas as pd
import numpy as np

## Reshaping by pivoting DataFrame objects

In [2]:
import pandas.util.testing as tm; tm.N = 3
def unpivot(frame):
    N, K = frame.shape
    data = {'value' : frame.values.ravel('F'),
            'variable' : np.asarray(frame.columns).repeat(N),
            'date' : np.tile(np.asarray(frame.index), K)}
    return pd.DataFrame(data, columns=['date', 'variable', 'value'])
df = unpivot(tm.makeTimeDataFrame())

In [3]:
df

,date,variable,value
0,2000-01-03,A,0.410458
1,2000-01-04,A,-0.109761
2,2000-01-05,A,-1.437635
3,2000-01-03,B,-0.424716
4,2000-01-04,B,0.510758
5,2000-01-05,B,0.139594
6,2000-01-03,C,-1.420203
7,2000-01-04,C,0.202483
8,2000-01-05,C,-0.207019
9,2000-01-03,D,2.467049


To select out everything for variable A we could do:

In [4]:
df[df['variable'] == 'A']

,date,variable,value
0,2000-01-03,A,0.410458
1,2000-01-04,A,-0.109761
2,2000-01-05,A,-1.437635


But suppose we wish to do time series operations with the variables. A better representation would be where the `columns` are the unique variables and an `index` of dates identifies individual observations. To reshape the data into this form, use the `pivot` function:

In [5]:
df.pivot(index = 'date', columns = 'variable', values = 'value')

variable,A,B,C,D
date,,,,
2000-01-03,0.410458,-0.424716,-1.420203,2.467049
2000-01-04,-0.109761,0.510758,0.202483,0.751175
2000-01-05,-1.437635,0.139594,-0.207019,-1.097861


If the `values` argument is omitted, and the input DataFrame has more than one column of values which are not used as column or index inputs to `pivot`, then the resulting “pivoted” DataFrame will have hierarchical columns whose topmost level indicates the respective value column:

In [6]:
df['value2'] = df['value'] * 2

In [7]:
df

,date,variable,value,value2
0,2000-01-03,A,0.410458,0.820916
1,2000-01-04,A,-0.109761,-0.219523
2,2000-01-05,A,-1.437635,-2.875270
3,2000-01-03,B,-0.424716,-0.849433
4,2000-01-04,B,0.510758,1.021516
5,2000-01-05,B,0.139594,0.279188
6,2000-01-03,C,-1.420203,-2.840407
7,2000-01-04,C,0.202483,0.404967
8,2000-01-05,C,-0.207019,-0.414038
9,2000-01-03,D,2.467049,4.934098


In [8]:
pivoted = df.pivot('date', 'variable')

In [9]:
pivoted

value                                  value2            \
variable           A         B         C         D         A         B   
date                                                                     
2000-01-03  0.410458 -0.424716 -1.420203  2.467049  0.820916 -0.849433   
2000-01-04 -0.109761  0.510758  0.202483  0.751175 -0.219523  1.021516   
2000-01-05 -1.437635  0.139594 -0.207019 -1.097861 -2.875270  0.279188   

                                
variable           C         D  
date                            
2000-01-03 -2.840407  4.934098  
2000-01-04  0.404967  1.502351  
2000-01-05 -0.414038 -2.195723

You of course can then select subsets from the pivoted DataFrame:

In [10]:
pivoted.loc[:,('value2','A')]  #pivoted['value2']['A']

date
2000-01-03    0.820916
2000-01-04   -0.219523
2000-01-05   -2.875270
Name: (value2, A), dtype: float64

Note that this returns a view on the underlying data in the case where the data are homogeneously-typed.

## Reshaping by stacking and unstacking

Closely related to the pivot function are the related stack and unstack functions currently available on Series and DataFrame. These functions are designed to work together with MultiIndex objects (see the section on hierarchical indexing). Here are essentially what these functions do:

* stack: “pivot” a level of the (possibly hierarchical) column labels, returning a DataFrame with an index with a new inner-most level of row labels.
* unstack: inverse operation from stack: “pivot” a level of the (possibly hierarchical) row index to the column axis, producing a reshaped DataFrame with a new inner-most level of column labels.

The clearest way to explain is by example. Let’s take a prior example data set from the hierarchical indexing section:

In [11]:
tuples = list(zip(*[['bar', 'bar', 'baz', 'baz',
                     'foo', 'foo', 'qux', 'qux'],
                    ['one', 'two', 'one', 'two',
                     'one', 'two', 'one', 'two']]))

In [13]:
tuples

[('bar', 'one'),
 ('bar', 'two'),
 ('baz', 'one'),
 ('baz', 'two'),
 ('foo', 'one'),
 ('foo', 'two'),
 ('qux', 'one'),
 ('qux', 'two')]

In [14]:
index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])

In [15]:
index

MultiIndex(levels=[[u'bar', u'baz', u'foo', u'qux'], [u'one', u'two']],
           labels=[[0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 0, 1, 0, 1, 0, 1]],
           names=[u'first', u'second'])

In [16]:
df = pd.DataFrame(np.random.randn(8, 2), index=index, columns=['A', 'B'])

In [17]:
df

A         B
first second                    
bar   one    -0.452777 -1.726914
      two     0.585278  0.595220
baz   one     0.086665 -1.135694
      two    -0.056205  1.069316
foo   one    -0.311231 -0.548153
      two    -0.282724  0.968075
qux   one     0.946619  0.881590
      two     1.524939  0.982270

In [18]:
df2 = df[:4]

In [19]:
df2

A         B
first second                    
bar   one    -0.452777 -1.726914
      two     0.585278  0.595220
baz   one     0.086665 -1.135694
      two    -0.056205  1.069316

The stack function “compresses” a level in the DataFrame’s columns to produce either:

* A Series, in the case of a simple column Index
* A DataFrame, in the case of a MultiIndex in the columns

If the columns have a MultiIndex, you can choose which level to stack. The stacked level becomes the new lowest level in a MultiIndex on the columns:

In [20]:
stacked = df2.stack()

In [21]:
stacked

first  second   
bar    one     A   -0.452777
               B   -1.726914
       two     A    0.585278
               B    0.595220
baz    one     A    0.086665
               B   -1.135694
       two     A   -0.056205
               B    1.069316
dtype: float64

With a “stacked” DataFrame or Series (having a MultiIndex as the index), the inverse operation of stack is unstack, which by default unstacks the last **level:**

In [22]:
stacked.unstack()

A         B
first second                    
bar   one    -0.452777 -1.726914
      two     0.585278  0.595220
baz   one     0.086665 -1.135694
      two    -0.056205  1.069316

In [23]:
 stacked.unstack(1)

second        one       two
first                      
bar   A -0.452777  0.585278
      B -1.726914  0.595220
baz   A  0.086665 -0.056205
      B -1.135694  1.069316

In [24]:
stacked.unstack(0)

first          bar       baz
second                      
one    A -0.452777  0.086665
       B -1.726914 -1.135694
two    A  0.585278 -0.056205
       B  0.595220  1.069316

If the indexes have names, you can use the level names instead of specifying the level numbers:

In [25]:
stacked.unstack('second')

second        one       two
first                      
bar   A -0.452777  0.585278
      B -1.726914  0.595220
baz   A  0.086665 -0.056205
      B -1.135694  1.069316

Notice that the stack and unstack methods implicitly sort the index levels involved. Hence a call to stack and then unstack, or viceversa, will result in a **sorted** copy of the original DataFrame or Series:

In [26]:
index = pd.MultiIndex.from_product([[2,1], ['a', 'b']])

In [27]:
index

MultiIndex(levels=[[1, 2], [u'a', u'b']],
           labels=[[1, 1, 0, 0], [0, 1, 0, 1]])

In [28]:
df = pd.DataFrame(np.random.randn(4), index=index, columns=['A'])

In [29]:
df

A
2 a  0.208557
  b -0.834462
1 a -0.493213
  b -1.309086

In [30]:
df.unstack().stack()

A
1 a -0.493213
  b -1.309086
2 a  0.208557
  b -0.834462

In [31]:
df.sort_index()

A
1 a -0.493213
  b -1.309086
2 a  0.208557
  b -0.834462

In [32]:
all(df.unstack().stack() == df.sort_index())

True

while the above code will raise a TypeError if the call to sort_index is removed.

### Multiple Levels

You may also stack or unstack more than one level at a time by passing a list of levels, in which case the end result is as if each level in the list were processed individually.

In [35]:
columns = pd.MultiIndex.from_tuples([
        ('A', 'cat', 'long'), ('B', 'cat', 'long'),
        ('A', 'dog', 'short'), ('B', 'dog', 'short')
                                    ],
                                    names=['exp', 'animal', 'hair_length']
                                    )
        

In [36]:
columns

MultiIndex(levels=[[u'A', u'B'], [u'cat', u'dog'], [u'long', u'short']],
           labels=[[0, 1, 0, 1], [0, 0, 1, 1], [0, 0, 1, 1]],
           names=[u'exp', u'animal', u'hair_length'])

In [37]:
df = pd.DataFrame(np.random.randn(4, 4), columns=columns)

In [38]:
df

exp,A,B,A,B
animal,cat,cat,dog,dog
hair_length,long,long,short,short
0,-0.473475,-0.676429,0.074029,-0.388624
1,0.328407,-1.460565,0.214722,1.684922
2,0.574179,0.912774,-0.883172,2.298887
3,-0.210455,-0.963509,0.634815,0.029889


In [39]:
df.stack(level=['animal', 'hair_length'])

exp                          A         B
  animal hair_length                    
0 cat    long        -0.473475 -0.676429
  dog    short        0.074029 -0.388624
1 cat    long         0.328407 -1.460565
  dog    short        0.214722  1.684922
2 cat    long         0.574179  0.912774
  dog    short       -0.883172  2.298887
3 cat    long        -0.210455 -0.963509
  dog    short        0.634815  0.029889

The list of levels can contain either level names or level numbers (but not a mixture of the two).

In [40]:
# df.stack(level=['animal', 'hair_length'])
# from above is equivalent to:
df.stack(level=[1, 2])

exp                          A         B
  animal hair_length                    
0 cat    long        -0.473475 -0.676429
  dog    short        0.074029 -0.388624
1 cat    long         0.328407 -1.460565
  dog    short        0.214722  1.684922
2 cat    long         0.574179  0.912774
  dog    short       -0.883172  2.298887
3 cat    long        -0.210455 -0.963509
  dog    short        0.634815  0.029889

### Missing Data

These functions are intelligent about handling missing data and do not expect each subgroup within the hierarchical index to have the same set of labels. They also can handle the index being unsorted (but you can make it sorted by calling sort_index, of course). Here is a more complex example:

In [42]:
columns = pd.MultiIndex.from_tuples([('A', 'cat'), ('B', 'dog'),
                                     ('B', 'cat'), ('A', 'dog')],
                                    names=['exp', 'animal'])
                                    
columns                     

MultiIndex(levels=[[u'A', u'B'], [u'cat', u'dog']],
           labels=[[0, 1, 1, 0], [0, 1, 0, 1]],
           names=[u'exp', u'animal'])

In [43]:
index = pd.MultiIndex.from_product([('bar', 'baz', 'foo', 'qux'),
                                    ('one', 'two')],
                                   names=['first', 'second'])
index
                                   
                                    

MultiIndex(levels=[[u'bar', u'baz', u'foo', u'qux'], [u'one', u'two']],
           labels=[[0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 0, 1, 0, 1, 0, 1]],
           names=[u'first', u'second'])

In [44]:
 df = pd.DataFrame(np.random.randn(8, 4), index=index, columns=columns)

In [45]:
df

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one    -0.922192 -0.699132 -0.663742 -0.417632
      two     0.208431  1.217991 -0.483664  0.062165
baz   one     0.198247  1.676166 -0.071215  0.541783
      two     0.624901  1.445567 -0.433575  0.185460
foo   one     0.283711 -0.180227 -0.626519 -1.288880
      two     0.700558  0.309803 -0.583970 -1.000960
qux   one     0.493563 -0.157331  0.225403  0.607988
      two     0.483183 -0.126232 -0.626745  0.307530

In [46]:
df2 = df.ix[[0, 1, 2, 4, 5, 7]]

In [47]:
df2

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one    -0.922192 -0.699132 -0.663742 -0.417632
      two     0.208431  1.217991 -0.483664  0.062165
baz   one     0.198247  1.676166 -0.071215  0.541783
foo   one     0.283711 -0.180227 -0.626519 -1.288880
      two     0.700558  0.309803 -0.583970 -1.000960
qux   two     0.483183 -0.126232 -0.626745  0.307530

As mentioned above, stack can be called with a level argument to select which level in the columns to stack:

In [48]:
df2.stack('exp')

animal                 cat       dog
first second exp                    
bar   one    A   -0.922192 -0.417632
             B   -0.663742 -0.699132
      two    A    0.208431  0.062165
             B   -0.483664  1.217991
baz   one    A    0.198247  0.541783
             B   -0.071215  1.676166
foo   one    A    0.283711 -1.288880
             B   -0.626519 -0.180227
      two    A    0.700558 -1.000960
             B   -0.583970  0.309803
qux   two    A    0.483183  0.307530
             B   -0.626745 -0.126232

In [49]:
df2.stack('animal')

exp                         A         B
first second animal                    
bar   one    cat    -0.922192 -0.663742
             dog    -0.417632 -0.699132
      two    cat     0.208431 -0.483664
             dog     0.062165  1.217991
baz   one    cat     0.198247 -0.071215
             dog     0.541783  1.676166
foo   one    cat     0.283711 -0.626519
             dog    -1.288880 -0.180227
      two    cat     0.700558 -0.583970
             dog    -1.000960  0.309803
qux   two    cat     0.483183 -0.626745
             dog     0.307530 -0.126232

Unstacking can result in missing values if subgroups do not have the same set of labels. By default, missing values will be replaced with the default fill value for that data type, NaN for float, NaT for datetimelike, etc. For integer types, by default data will converted to float and missing values will be set to NaN.

In [50]:
df3 = df.iloc[[0, 1, 4, 7], [1, 2]]

In [51]:
df3

exp                  B          
animal             dog       cat
first second                    
bar   one    -0.699132 -0.663742
      two     1.217991 -0.483664
foo   one    -0.180227 -0.626519
qux   two    -0.126232 -0.626745

In [52]:
df3.unstack()

exp            B                              
animal       dog                 cat          
second       one       two       one       two
first                                         
bar    -0.699132  1.217991 -0.663742 -0.483664
foo    -0.180227       NaN -0.626519       NaN
qux          NaN -0.126232       NaN -0.626745

Alternatively, unstack takes an optional fill_value argument, for specifying the value of missing data.

In [53]:
df3.unstack(fill_value= 'MISSING')

exp            B                              
animal       dog                 cat          
second       one       two       one       two
first                                         
bar    -0.699132   1.21799 -0.663742 -0.483664
foo    -0.180227   MISSING -0.626519   MISSING
qux      MISSING -0.126232   MISSING -0.626745

### With a MultiIndex

Unstacking when the columns are a MultiIndex is also careful about doing the right thing:

In [54]:
df[:3].unstack(0)

exp            A                   B                                       A  \
animal       cat                 dog                 cat                 dog   
first        bar       baz       bar       baz       bar       baz       bar   
second                                                                         
one    -0.922192  0.198247 -0.699132  1.676166 -0.663742 -0.071215 -0.417632   
two     0.208431       NaN  1.217991       NaN -0.483664       NaN  0.062165   

exp               
animal            
first        baz  
second            
one     0.541783  
two          NaN

In [55]:
df2.unstack(1)

exp            A                   B                                       A  \
animal       cat                 dog                 cat                 dog   
second       one       two       one       two       one       two       one   
first                                                                          
bar    -0.922192  0.208431 -0.699132  1.217991 -0.663742 -0.483664 -0.417632   
baz     0.198247       NaN  1.676166       NaN -0.071215       NaN  0.541783   
foo     0.283711  0.700558 -0.180227  0.309803 -0.626519 -0.583970 -1.288880   
qux          NaN  0.483183       NaN -0.126232       NaN -0.626745       NaN   

exp               
animal            
second       two  
first             
bar     0.062165  
baz          NaN  
foo    -1.000960  
qux     0.307530

## Reshaping by Melt

The melt() function is useful to massage a DataFrame into a format where one or more columns are identifier variables, while all other columns, considered measured variables, are “unpivoted” to the row axis, leaving just two non-identifier columns, “variable” and “value”. The names of those columns can be customized by supplying the var_name and value_name parameters.

For instance,

In [56]:
cheese = pd.DataFrame({'first' : ['John', 'Mary'],
                       'last' : ['Doe', 'Bo'],
                       'height' : [5.5, 6.0],
                       'weight' : [130, 150]})

In [57]:
cheese

,first,height,last,weight
0,John,5.5,Doe,130
1,Mary,6.0,Bo,150


In [58]:
pd.melt(cheese, id_vars = ['first', 'last'])

,first,last,variable,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [59]:
pd.melt(cheese, id_vars=['first', 'last'], var_name='quantity')

,first,last,quantity,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


Another way to transform is to use the wide_to_long panel data convenience function.

In [60]:
dft = pd.DataFrame({"A1970" : {0 : "a", 1 : "b", 2 : "c"},
                    "A1980" : {0 : "d", 1 : "e", 2 : "f"},
                    "B1970" : {0 : 2.5, 1 : 1.2, 2 : .7},
                    "B1980" : {0 : 3.2, 1 : 1.3, 2 : .1},
                    "X"     : dict(zip(range(3), np.random.randn(3)))
                    })

In [61]:
dft

,A1970,A1980,B1970,B1980,X
0,a,d,2.5,3.2,0.490023
1,b,e,1.2,1.3,-2.073865
2,c,f,0.7,0.1,-0.295792


In [62]:
dft["id"] = dft.index

In [63]:
dft

,A1970,A1980,B1970,B1980,X,id
0,a,d,2.5,3.2,0.490023,0
1,b,e,1.2,1.3,-2.073865,1
2,c,f,0.7,0.1,-0.295792,2


In [64]:
pd.wide_to_long(dft, ["A", "B"], i="id", j="year")

,,X,A,B
id,year,,,
0,1970,0.490023,a,2.5
1,1970,-2.073865,b,1.2
2,1970,-0.295792,c,0.7
0,1980,0.490023,d,3.2
1,1980,-2.073865,e,1.3
2,1980,-0.295792,f,0.1


## Combining with stats and GroupBy

It should be no shock that combining pivot / stack / unstack with GroupBy and the basic Series and DataFrame statistical functions can produce some very expressive and fast data manipulations.

In [65]:
df

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one    -0.922192 -0.699132 -0.663742 -0.417632
      two     0.208431  1.217991 -0.483664  0.062165
baz   one     0.198247  1.676166 -0.071215  0.541783
      two     0.624901  1.445567 -0.433575  0.185460
foo   one     0.283711 -0.180227 -0.626519 -1.288880
      two     0.700558  0.309803 -0.583970 -1.000960
qux   one     0.493563 -0.157331  0.225403  0.607988
      two     0.483183 -0.126232 -0.626745  0.307530

In [72]:
df.stack()

exp                         A         B
first second animal                    
bar   one    cat    -0.922192 -0.663742
             dog    -0.417632 -0.699132
      two    cat     0.208431 -0.483664
             dog     0.062165  1.217991
baz   one    cat     0.198247 -0.071215
             dog     0.541783  1.676166
      two    cat     0.624901 -0.433575
             dog     0.185460  1.445567
foo   one    cat     0.283711 -0.626519
             dog    -1.288880 -0.180227
      two    cat     0.700558 -0.583970
             dog    -1.000960  0.309803
qux   one    cat     0.493563  0.225403
             dog     0.607988 -0.157331
      two    cat     0.483183 -0.626745
             dog     0.307530 -0.126232

In [76]:
df.stack().mean(1)

first  second  animal
bar    one     cat      -0.792967
               dog      -0.558382
       two     cat      -0.137617
               dog       0.640078
baz    one     cat       0.063516
               dog       1.108975
       two     cat       0.095663
               dog       0.815514
foo    one     cat      -0.171404
               dog      -0.734553
       two     cat       0.058294
               dog      -0.345579
qux    one     cat       0.359483
               dog       0.225328
       two     cat      -0.071781
               dog       0.090649
dtype: float64

In [77]:
df.stack().mean(1).unstack()

animal             cat       dog
first second                    
bar   one    -0.792967 -0.558382
      two    -0.137617  0.640078
baz   one     0.063516  1.108975
      two     0.095663  0.815514
foo   one    -0.171404 -0.734553
      two     0.058294 -0.345579
qux   one     0.359483  0.225328
      two    -0.071781  0.090649

In [85]:
list(df.groupby(level = 1, axis = 1))

[('cat', exp                  A         B
  animal             cat       cat
  first second                    
  bar   one    -0.922192 -0.663742
        two     0.208431 -0.483664
  baz   one     0.198247 -0.071215
        two     0.624901 -0.433575
  foo   one     0.283711 -0.626519
        two     0.700558 -0.583970
  qux   one     0.493563  0.225403
        two     0.483183 -0.626745), ('dog', exp                  B         A
  animal             dog       dog
  first second                    
  bar   one    -0.699132 -0.417632
        two     1.217991  0.062165
  baz   one     1.676166  0.541783
        two     1.445567  0.185460
  foo   one    -0.180227 -1.288880
        two     0.309803 -1.000960
  qux   one    -0.157331  0.607988
        two    -0.126232  0.307530)]

In [87]:
# same result, another way
df.groupby(level=1, axis=1).mean()

animal             cat       dog
first second                    
bar   one    -0.792967 -0.558382
      two    -0.137617  0.640078
baz   one     0.063516  1.108975
      two     0.095663  0.815514
foo   one    -0.171404 -0.734553
      two     0.058294 -0.345579
qux   one     0.359483  0.225328
      two    -0.071781  0.090649

In [89]:
df.stack().groupby(level=1, axis = 0).mean()

exp,A,B
second,,
one,-0.062926,-0.062075
two,0.196408,0.089897


In [93]:
df.mean().unstack(0)

exp,A,B
animal,,
cat,0.258800,-0.408004
dog,-0.125318,0.435826


## Pivot tables

The function pandas.pivot_table can be used to create spreadsheet-style pivot tables. See the cookbook for some advanced strategies

It takes a number of arguments

* data: A DataFrame object
* values: a column or a list of columns to aggregate
* index: a column, Grouper, array which has the same length as data, or list of them. Keys to group by on the pivot table index. If an array is passed, it is being used as the same manner as column values.
* columns: a column, Grouper, array which has the same length as data, or list of them. Keys to group by on the pivot table column. If an array is passed, it is being used as the same manner as column values.
* aggfunc: function to use for aggregation, defaulting to numpy.mean

Consider a data set like this:

In [94]:
import datetime

In [95]:
df = pd.DataFrame({'A': ['one', 'one', 'two', 'three'] * 6,
                   'B': ['A', 'B', 'C'] * 8,
                   'C': ['foo', 'foo', 'foo', 'bar', 'bar', 'bar'] * 4,
                   'D': np.random.randn(24),
                   'E': np.random.randn(24),
                   'F': [datetime.datetime(2013, i, 1) for i in range(1, 13)] +
                   [datetime.datetime(2013, i, 15) for i in range(1, 13)]})

In [96]:
df

,A,B,C,D,E,F
0,one,A,foo,-0.183710,0.933656,2013-01-01
1,one,B,foo,0.375287,-0.816964,2013-02-01
2,two,C,foo,-0.333964,-1.750032,2013-03-01
3,three,A,bar,-0.913241,0.616258,2013-04-01
4,one,B,bar,0.592812,1.366945,2013-05-01
5,one,C,bar,0.904336,-0.195462,2013-06-01
6,two,A,foo,1.538698,-0.272382,2013-07-01
7,three,B,foo,0.875376,0.920065,2013-08-01
8,one,C,foo,0.440583,-1.119335,2013-09-01
9,one,A,bar,0.064746,-0.490524,2013-10-01


We can produce pivot tables from this data very easily:

In [97]:
pd.pivot_table(df, values = 'D', index = ['A', 'B'], columns = ['C'])

C             bar       foo
A     B                    
one   A -0.512221  0.181204
      B  0.722071  0.693705
      C  1.400229  0.218224
three A -0.343383       NaN
      B       NaN  0.034139
      C  0.771261       NaN
two   A       NaN  0.741712
      B -0.621636       NaN
      C       NaN  0.070556

In [98]:
pd.pivot_table(df, values = 'D', index = ['B'], columns = ['A', 'C'], aggfunc = np.sum)

A       one               three                 two          
C       bar       foo       bar       foo       bar       foo
B                                                            
A -1.024441  0.362408 -0.686766       NaN       NaN  1.483425
B  1.444142  1.387411       NaN  0.068279 -1.243273       NaN
C  2.800459  0.436449  1.542522       NaN       NaN  0.141111

In [99]:
pd.pivot_table(df, values=['D','E'], index=['B'], columns=['A', 'C'], aggfunc=np.sum)

D                                                           E  \
A       one               three                 two                 one   
C       bar       foo       bar       foo       bar       foo       bar   
B                                                                         
A -1.024441  0.362408 -0.686766       NaN       NaN  1.483425  0.311711   
B  1.444142  1.387411       NaN  0.068279 -1.243273       NaN  1.201436   
C  2.800459  0.436449  1.542522       NaN       NaN  0.141111 -1.527953   

                                                     
A               three                 two            
C       foo       bar       foo       bar       foo  
B                                                    
A  3.023201  0.394762       NaN       NaN -0.160141  
B  0.584751       NaN  1.162193  2.484877       NaN  
C -0.220964  0.523780       NaN       NaN -1.613905

The result object is a DataFrame having potentially hierarchical indexes on the rows and columns. If the values column name is not given, the pivot table will include all of the data that can be aggregated in an additional level of hierarchy in the columns:

In [100]:
 pd.pivot_table(df, index=['A', 'B'], columns=['C'])

D                   E          
C             bar       foo       bar       foo
A     B                                        
one   A -0.512221  0.181204  0.155856  1.511600
      B  0.722071  0.693705  0.600718  0.292376
      C  1.400229  0.218224 -0.763976 -0.110482
three A -0.343383       NaN  0.197381       NaN
      B       NaN  0.034139       NaN  0.581097
      C  0.771261       NaN  0.261890       NaN
two   A       NaN  0.741712       NaN -0.080070
      B -0.621636       NaN  1.242438       NaN
      C       NaN  0.070556       NaN -0.806952

Also, you can use Grouper for index and columns keywords. For detail of Grouper, see [Grouping with a Grouper specification](http://pandas.pydata.org/pandas-docs/stable/groupby.html#groupby-specify).

In [101]:
pd.pivot_table(df, values='D', index=pd.Grouper(freq='M', key='F'), columns='C')

C,bar,foo
F,,
2013-01-31,NaN,0.181204
2013-02-28,NaN,0.693705
2013-03-31,NaN,0.070556
2013-04-30,-0.343383,NaN
2013-05-31,0.722071,NaN
2013-06-30,1.400229,NaN
2013-07-31,NaN,0.741712
2013-08-31,NaN,0.034139
2013-09-30,NaN,0.218224


You can render a nice output of the table omitting the missing values by calling to_string if you wish:

In [102]:
table = pd.pivot_table(df, index=['A', 'B'], columns=['C'])

In [103]:
table

D                   E          
C             bar       foo       bar       foo
A     B                                        
one   A -0.512221  0.181204  0.155856  1.511600
      B  0.722071  0.693705  0.600718  0.292376
      C  1.400229  0.218224 -0.763976 -0.110482
three A -0.343383       NaN  0.197381       NaN
      B       NaN  0.034139       NaN  0.581097
      C  0.771261       NaN  0.261890       NaN
two   A       NaN  0.741712       NaN -0.080070
      B -0.621636       NaN  1.242438       NaN
      C       NaN  0.070556       NaN -0.806952

In [104]:
print(table.to_string(na_rep=''))

                D                   E          
C             bar       foo       bar       foo
A     B                                        
one   A -0.512221  0.181204  0.155856  1.511600
      B  0.722071  0.693705  0.600718  0.292376
      C  1.400229  0.218224 -0.763976 -0.110482
three A -0.343383            0.197381          
      B            0.034139            0.581097
      C  0.771261            0.261890          
two   A            0.741712           -0.080070
      B -0.621636            1.242438          
      C            0.070556           -0.806952


Note that pivot_table is also available as an instance method on DataFrame.

### Adding margins

If you pass margins=True to pivot_table, special All columns and rows will be added with partial group aggregates across the categories on the rows and columns:

In [105]:
 df.pivot_table(index=['A', 'B'], columns='C', margins=True, aggfunc=np.std)

D                             E                    
C             bar       foo       All       bar       foo       All
A     B                                                            
one   A  0.815954  0.516066  0.686281  0.914119  0.817337  1.055414
      B  0.182799  0.450312  0.281070  1.083609  1.568843  1.115131
      C  0.701299  0.314463  0.814011  0.804001  1.426734  1.018011
three A  0.805900       NaN  0.805900  0.592381       NaN  0.592381
      B       NaN  1.189688  1.189688       NaN  0.479374  0.479374
      C  1.185535       NaN  1.185535  0.331482       NaN  0.331482
two   A       NaN  1.127108  1.127108       NaN  0.271970  0.271970
      B  0.703394       NaN  0.703394  0.634739       NaN  0.634739
      C       NaN  0.572078  0.572078       NaN  1.333715  1.333715
All      0.988997  0.644063  0.800205  0.842241  1.102123  0.939418

## Cross tabulations

Use the crosstab function to compute a cross-tabulation of two (or more) factors. By default crosstab computes a frequency table of the factors unless an array of values and an aggregation function are passed.

It takes a number of arguments

* index: array-like, values to group by in the rows
* columns: array-like, values to group by in the columns
* values: array-like, optional, array of values to aggregate according to the factors
* aggfunc: function, optional, If no values array is passed, computes a frequency table
* rownames: sequence, default None, must match number of row arrays passed
* colnames: sequence, default None, if passed, must match number of column arrays passed
* margins: boolean, default False, Add row/column margins (subtotals)
* normalize: boolean, {‘all’, ‘index’, ‘columns’}, or {0,1}, default False. Normalize by dividing all values by the sum of values.

Any Series passed will have their name attributes used unless row or column names for the cross-tabulation are specified

For example:

In [106]:
foo, bar, dull, shiny, one, two = 'foo', 'bar', 'dull', 'shiny', 'one', 'two'

In [107]:
a = np.array([foo, foo, bar, bar, foo, foo], dtype=object)

In [108]:
a

array(['foo', 'foo', 'bar', 'bar', 'foo', 'foo'], dtype=object)

In [109]:
b = np.array([one, one, two, one, two, one], dtype=object)

In [110]:
c = np.array([dull, dull, shiny, dull, dull, shiny], dtype=object)

In [111]:
pd.crosstab(a, [b, c], rownames=['a'], colnames=['b', 'c'])

b    one        two      
c   dull shiny dull shiny
a                        
bar    1     0    0     1
foo    2     1    1     0

If crosstab receives only two Series, it will provide a frequency table.

In [112]:
df = pd.DataFrame({'A': [1, 2, 2, 2, 2], 'B': [3, 3, 4, 4, 4],
                   'C': [1, 1, np.nan, 1, 1]})

In [113]:
df

,A,B,C
0,1,3,1.0
1,2,3,1.0
2,2,4,NaN
3,2,4,1.0
4,2,4,1.0


In [126]:
pd.crosstab(df.A, df.B)

B,3,4
A,,
1,1,0
2,1,3


In [127]:
pd.crosstab(df.B, df.A)

A,1,2
B,,
3,1,1
4,0,3


### Normalization

Frequency tables can also be normalized to show percentages rather than counts using the normalize argument:

In [115]:
pd.crosstab(df.A, df.B, normalize=True)

B,3,4
A,,
1,0.2,0.0
2,0.2,0.6


normalize can also normalize values within each row or within each column:

In [116]:
 pd.crosstab(df.A, df.B, normalize='columns')

B,3,4
A,,
1,0.5,0.0
2,0.5,1.0


In [121]:
 pd.crosstab(df.A, df.B, normalize= 0) #rows

B,3,4
A,,
1,1.00,0.00
2,0.25,0.75


crosstab can also be passed a third Series and an aggregation function (aggfunc) that will be applied to the values of the third Series within each group defined by the first two Series:

In [124]:
pd.crosstab(df.A, df.B, values=df.C, aggfunc=np.sum)

B,3,4
A,,
1,1.0,NaN
2,1.0,2.0


### Adding Margins

Finally, one can also add margins or normalize this output.

In [128]:
pd.crosstab(df.A, df.B, values=df.C, aggfunc=np.sum, normalize=True,
            margins=True)

B,3,4,All
A,,,
1,0.25,0.0,0.25
2,0.25,0.5,0.75
All,0.50,0.5,1.00


## Tiling

The cut function computes groupings for the values of the input array and is often used to transform continuous variables to discrete or categorical variables:

In [129]:
ages = np.array([10, 15, 13, 12, 23, 25, 28, 59, 60])

In [130]:
ages

array([10, 15, 13, 12, 23, 25, 28, 59, 60])

In [131]:
pd.cut(ages, bins=3)

[(9.95, 26.667], (9.95, 26.667], (9.95, 26.667], (9.95, 26.667], (9.95, 26.667], (9.95, 26.667], (26.667, 43.333], (43.333, 60], (43.333, 60]]
Categories (3, object): [(9.95, 26.667] < (26.667, 43.333] < (43.333, 60]]

If the bins keyword is an integer, then equal-width bins are formed. Alternatively we can specify custom bin-edges:

In [132]:
 pd.cut(ages, bins=[0, 18, 35, 70])

[(0, 18], (0, 18], (0, 18], (0, 18], (18, 35], (18, 35], (18, 35], (35, 70], (35, 70]]
Categories (3, object): [(0, 18] < (18, 35] < (35, 70]]

## Computing indicator / dummy variables

To convert a categorical variable into a “dummy” or “indicator” DataFrame, for example a column in a DataFrame (a Series) which has k distinct values, can derive a DataFrame containing k columns of 1s and 0s:

In [133]:
 df = pd.DataFrame({'key': list('bbacab'), 'data1': range(6)})

In [134]:
df

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [135]:
pd.get_dummies(df['key'])

,a,b,c
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0


Sometimes it’s useful to prefix the column names, for example when merging the result with the original DataFrame:

In [136]:
dummies = pd.get_dummies(df['key'], prefix='key')

In [137]:
dummies

,key_a,key_b,key_c
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0


In [138]:
df[['data1']].join(dummies)

,data1,key_a,key_b,key_c
0,0,0.0,1.0,0.0
1,1,0.0,1.0,0.0
2,2,1.0,0.0,0.0
3,3,0.0,0.0,1.0
4,4,1.0,0.0,0.0
5,5,0.0,1.0,0.0


This function is often used along with discretization functions like cut:

In [139]:
values = np.random.randn(10)

In [140]:
values

array([-1.1069359 , -1.4817102 , -1.63367552,  1.30804103,  1.2764695 ,
        0.35499255, -0.48202605, -0.11880869,  1.35114896, -0.81999232])

In [141]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

In [142]:
pd.get_dummies(pd.cut(values, bins))

,"(0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1]"
0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0
5,0.0,1.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0


get_dummies() also accepts a DataFrame. By default all categorical variables (categorical in the statistical sense, those with object or categorical dtype) are encoded as dummy variables.

In [143]:
df = pd.DataFrame({'A': ['a', 'b', 'a'], 'B': ['c', 'c', 'b'],
                   'C': [1, 2, 3]})

In [144]:
df

,A,B,C
0,a,c,1
1,b,c,2
2,a,b,3


In [145]:
pd.get_dummies(df)

,C,A_a,A_b,B_b,B_c
0,1,1.0,0.0,0.0,1.0
1,2,0.0,1.0,0.0,1.0
2,3,1.0,0.0,1.0,0.0


All non-object columns are included untouched in the output.

You can control the columns that are encoded with the columns keyword.

In [146]:
pd.get_dummies(df, columns=['A'])

,B,C,A_a,A_b
0,c,1,1.0,0.0
1,c,2,0.0,1.0
2,b,3,1.0,0.0


Notice that the B column is still included in the output, it just hasn’t been encoded. You can drop B before calling get_dummies if you don’t want to include it in the output.

As with the Series version, you can pass values for the prefix and prefix_sep. By default the column name is used as the prefix, and ‘_’ as the prefix separator. You can specify prefix and prefix_sep in 3 ways

* string: Use the same value for prefix or prefix_sep for each column to be encoded
* list: Must be the same length as the number of columns being encoded.
* dict: Mapping column name to prefix

In [147]:
simple = pd.get_dummies(df, prefix='new_prefix')

In [148]:
simple

,C,new_prefix_a,new_prefix_b,new_prefix_b,new_prefix_c
0,1,1.0,0.0,0.0,1.0
1,2,0.0,1.0,0.0,1.0
2,3,1.0,0.0,1.0,0.0


In [149]:
from_list = pd.get_dummies(df, prefix=['from_A', 'from_B'])

In [150]:
from_list

,C,from_A_a,from_A_b,from_B_b,from_B_c
0,1,1.0,0.0,0.0,1.0
1,2,0.0,1.0,0.0,1.0
2,3,1.0,0.0,1.0,0.0


In [152]:
from_dict = pd.get_dummies(df, prefix={'B': 'from_B', 'A': 'from_A'})

In [153]:
from_dict

,C,from_A_a,from_A_b,from_B_b,from_B_c
0,1,1.0,0.0,0.0,1.0
1,2,0.0,1.0,0.0,1.0
2,3,1.0,0.0,1.0,0.0


Sometimes it will be useful to only keep k-1 levels of a categorical variable to avoid collinearity when feeding the result to statistical models. You can switch to this mode by turn on drop_first.

In [154]:
s = pd.Series(list('abcaa'))

In [155]:
s

0    a
1    b
2    c
3    a
4    a
dtype: object

In [156]:
pd.get_dummies(s)

,a,b,c
0,1.0,0.0,0.0
1,0.0,1.0,0.0
2,0.0,0.0,1.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0


In [157]:
pd.get_dummies(s, drop_first=True)

,b,c
0,0.0,0.0
1,1.0,0.0
2,0.0,1.0
3,0.0,0.0
4,0.0,0.0


When a column contains only one level, it will be omitted in the result.

In [158]:
df = pd.DataFrame({'A':list('aaaaa'),'B':list('ababc')})

In [159]:
pd.get_dummies(df)

,A_a,B_a,B_b,B_c
0,1.0,1.0,0.0,0.0
1,1.0,0.0,1.0,0.0
2,1.0,1.0,0.0,0.0
3,1.0,0.0,1.0,0.0
4,1.0,0.0,0.0,1.0


In [160]:
pd.get_dummies(df, drop_first=True)

,B_b,B_c
0,0.0,0.0
1,1.0,0.0
2,0.0,0.0
3,1.0,0.0
4,0.0,1.0


## Factorizing values

To encode 1-d values as an enumerated type use factorize:

In [161]:
x = pd.Series(['A', 'A', np.nan, 'B', 3.14, np.inf])

In [162]:
x

0       A
1       A
2     NaN
3       B
4    3.14
5     inf
dtype: object

In [163]:
labels, uniques = pd.factorize(x)

In [164]:
labels

array([ 0,  0, -1,  1,  2,  3])

In [165]:
uniques

Index([u'A', u'B', 3.14, inf], dtype='object')

Note that factorize is similar to numpy.unique, but differs in its handling of NaN:

In [166]:
pd.factorize(x, sort=True)

(array([ 2,  2, -1,  3,  0,  1]),
 Index([3.14, inf, u'A', u'B'], dtype='object'))

In [167]:
np.unique(x, return_inverse=True)[::-1]

(array([3, 3, 0, 4, 1, 2], dtype=int64),
 array([nan, 3.14, inf, 'A', 'B'], dtype=object))